# **Clone the repository:**

If already public:

In [ ]:
!git clone https://github.com/adsmendesdaniel/PyTurbulence.git

If not, generate a token number on Github (Settings -> Developer settings -> Personal access tokens -> Tokens (Classic), and substitute its code on the line below:

In [8]:
!git clone https://YOUR_TOKEN_HERE@github.com/adsmendesdaniel/PyTurbulence.git

Cloning into 'PyTurbulence'...
remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 7 (delta 1), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (7/7), 13.81 KiB | 2.30 MiB/s, done.
Resolving deltas: 100% (1/1), done.


In [1]:
!git clone https://@github.com/adsmendesdaniel/PyTurbulence.git

Cloning into 'PyTurbulence'...
remote: Enumerating objects: 22, done.
remote: Counting objects: 100% (22/22), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 22 (delta 7), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (22/22), 17.44 KiB | 17.44 MiB/s, done.
Resolving deltas: 100% (7/7), done.


# **Install and/or import libraries:**

In [16]:
#import gc

#import numpy as np
#import time as time
#import pickle as pkl
#import scipy.interpolate

#import matplotlib.pyplot as plt
#from matplotlib.colors import Normalize
#from matplotlib.cm import ScalarMappable


bash: cannot set terminal process group (155): Inappropriate ioctl for device
bash: no job control in this shell
/content# !rm -rf PyTurbulence
bash: !rm: event not found
/content# rm -rf PyTurbulence



/content# ^C


# **Simulation:**

## Functions:

## Plot function:

In [2]:
import seaborn as sns
sns.set(style='dark')

def plot(P, u, v, U, curl, t, T, step, Nstep):
    '''
    # Adjust subplot layout
    fig.subplots_adjust(
        left=0.05,  # Left margin
        right=0.95,  # Right margin
        top=0.9,  # Top margin
        bottom=0.1,  # Bottom margin
        wspace=0.3,  # Space between subplots (width)
        hspace=0.  # Space between subplots (height)
    )
    '''
    # Colormap for plots
    cmap = plt.cm.jet  # Colormap to use

    # List of datasets and titles
    data_list = [U, curl] #[P, np.linalg.norm(U, axis=0, keepdims=True)[0], np.linalg.norm(curl, axis=0, keepdims=True)[0]]
    titles = ['Velocity', 'Vorticity'] #['Pressão', 'Velocidade', 'Vorticidade']

    # Create a 1xN grid of plots
    fig, axs = plt.subplots(1, len(titles), figsize=(15, 7), dpi=200, num=1, clear=True)
    fig.suptitle(r'$\mathbf{Physical\ Time:}$ ' + f'{t:.2f}/{T}  |  ' + r'$\mathbf{Iteration:}$ ' + f'{step}/{Nstep}', fontsize=16)

    # Iterate over subplots to plot data with individual colorbars
    for ax, data, titulo in zip(axs, data_list, titles):

        # Normalization for the colorbar
        vmin = data.min()
        vmax = data.max()
        norm = Normalize(vmin=vmin, vmax=vmax)

        ax.imshow(data, extent=[0, Lx, 0, Ly],
                  cmap=cmap, norm=norm, aspect='equal', interpolation='bilinear')

        ax.set_title(titulo, fontweight='bold', fontstyle='italic')
        ax.set_xlabel("X", fontweight='bold')
        ax.set_ylabel("Y", fontweight='bold')

        # Create a ScalarMappable for the colorbar for this subplot
        mappable = ScalarMappable(norm=norm, cmap=cmap)
        mappable.set_array(data)

        # Add the individual colorbar to this subplot
        #fig.colorbar(mappable, ax=ax, orientation='vertical', shrink=0.6, pad=0.15, label="Norma da vorticidade")

    # Save the figure
    nome_tmp = 'plot_'+str(step)+'.png'
    lista_enderecos.append(nome_tmp)
    plt.savefig('/content/'+nome_tmp)
    plt.close(fig)
    #gc.collect()
    #plt.show()

#List to save plots:
lista_enderecos = []


In [ ]:
#from time import time

#from matplotlib.colors import Normalize
#from matplotlib.cm import ScalarMappable
#import noise

#from numpy.random import seed, uniform
#from numpy import max as npmax
#import matplotlib.animation as animation
#from basic_units import radians, degrees, cos
#from radians_plot import *

# parent = os.path.abspath(os.path.join(os.path.dirname(__file__), '.'))
# sys.path.append(parent)


## Main Loop:

In [54]:
try:
    from mpi4py import MPI
except ImportError:
    !pip install mpi4py
    from mpi4py import MPI

%run PyTurbulence/Spectral_DNS/initial_conditions.py #Import initial_conditions is not working on colab

import time
import numpy as np
import matplotlib.pyplot as plt

from tqdm import tqdm
from numpy.fft import fftfreq, fft, ifft, fft2, ifft2, fftshift, ifftshift

#from PyTurbulence.Spectral_DNS.initial_conditions import IC_2D

#Initialize x, y kx, ky coordinate
def IC_coor(Nx, Ny, Np, dx, dy, rank, num_processes):

    x = np.zeros((Np, Ny), dtype=float);
    y = np.zeros((Np, Ny), dtype=float);
    kx = np.zeros((Nx, Np), dtype=float);
    ky = np.zeros((Nx, Np), dtype=float);

    for j in range(Ny):
        x[0:Np, j] = range(Np);
        if num_processes == 1:
            x[0:Nx, j] = range(int(-Nx / 2), int(Nx / 2));

    # offset for mpi
    if num_processes != 1:
        x = x - (num_processes / 2 - rank) * Np

    x = x * dx;

    for i in range(Np):
        y[i, 0:Ny] = range(int(-Ny / 2), int(Ny / 2));

    y = y * dy;

    for j in range(Np):
        kx[0:Nx, j] = range(int(-Nx / 2), int(Nx / 2));

    for i in range(Nx):
        ky[i, 0:Np] = range(Np);

        if num_processes == 1:
            ky[i, 0:Ny] = range(int(-Ny / 2), int(Ny / 2));

    # offset for mpi
    if num_processes != 1:
        ky = ky - (num_processes / 2 - rank) * Np

    k2 = kx ** 2 + ky ** 2;

    for i in range(Nx):
        for j in range(Np):
            if (k2[i, j] == 0):
                k2[i, j] = 1e-5;  # so that I do not divide by 0 below when using
            # projection operator

    # k2_exp = exp(-nu * (k2 ** 5) * dt - nu_hypo * dt);
    k2_inv = K2_inv = 1/np.where(k2 == 0, 1, k2).astype(float)

    return x, y, kx, ky, k2, k2_inv

def fftn_mpi(u, fu):
    Uc_hatT[:] = fftshift(fft(ifftshift(u), axis=1))
    U_mpi[:] = rollaxis(Uc_hatT.reshape(Np, num_processes, Np), 1)
    comm.Alltoall([U_mpi, MPI.DOUBLE_COMPLEX], [fu, MPI.DOUBLE_COMPLEX])
    fu[:] = fftshift(fft(ifftshift(fu), axis=0))
    return fu

def ifftn_mpi(fu, u):
    Uc_hat[:] = ifftshift(ifft(fftshift(fu), axis=0))
    comm.Alltoall([Uc_hat, MPI.DOUBLE_COMPLEX], [U_mpi, MPI.DOUBLE_COMPLEX])
    Uc_hatT[:] = rollaxis(U_mpi, 1).reshape(Uc_hatT.shape)
    u[:] = ifftshift(ifft(fftshift(Uc_hatT), axis=1))
    return u

# SIMULATION PARAMETERS:
viscosity = 1/16000      # Viscosity = 1/Reynolds. Suggestion: Reynolds -> 1600 or 16000
t_f = 300                # Final physical time
dt = 0.005               # Time step. Suggestion: 0.01 or 0.001
N = Nx = Ny = 2**9       # Grid dimension
IC_type = 'taylor_green' # Initial conditions. Check initial_conditions.py

# MPI SETUP:
comm = MPI.COMM_WORLD
num_processes = comm.Get_size()
rank = comm.Get_rank()
Np = int(N/num_processes) #Slab decomposition
print('num_processes', num_processes, 'rank', rank)

n_steps = int(np.ceil(t_f/dt)) #Number of frames in the simulatino

# Spatial domain parameters
Lx = 2*np.pi;
Ly = 2*np.pi;
dx = Lx/Nx;
dy = Ly/Ny;



#Coordinates and wave numbers
x, y, Kx, Ky, K2, K2_inv = IC_coor(Nx, Ny, Np, dx, dy, rank, num_processes) # TODO check what needs to be done with the wave numbers to get rid of IC_Coor function
sx = slice(rank*Np,(rank+1)*Np)
Xmesh = np.mgrid[sx, :N].astype(float) * Lx / N
X = Xmesh[0]
Y = Xmesh[1]



#Uc_hat = empty((N, Np), dtype=complex)
Uc_hatT = np.empty((Np, N), dtype=complex)
U_mpi = np.empty((num_processes, Np, Np), dtype=complex)



x = Y[0] #Is this really being used?
y = Y[0] #Is this really being used?
kx = fftfreq(N, 1./N)
ky = kx.copy()
''' I dont know why this exists...
K = array(meshgrid(kx, ky[sx], indexing='ij'), dtype=int)
Kx = K[1]
Ky = K[0]
K2 = sum(K * K, 0, dtype=int)
'''
LapHat = K2.copy()
LapHat *= -1
#K2[0][0] = 1
K2 *=-1
K2_inv = 1 / np.where(K2 == 0, 1, K2).astype(float)
ikx_over_K2 = 1j * Kx * K2_inv
iky_over_K2 = 1j * Ky * K2_inv



# Define dealias:
kmax_dealias = 2./3.*(N/2 + 1)
dealias = np.array((abs(Kx) < kmax_dealias)*(abs(Ky) < kmax_dealias), dtype=bool)



# Preallocate arrays
u_hat = np.zeros((Nx, Np), dtype=complex)
v_hat = np.zeros((Nx, Np), dtype=complex)
u = np.zeros((Np, Ny), dtype=float)
v = np.zeros((Np, Ny), dtype=float)
#omega_hat0 = np.zeros((Nx, Np), dtype=complex)
#omega_hat1 = np.zeros((Nx, Np), dtype=complex)
omega_hat = np.zeros((Nx, Np), dtype=complex)
#omega_hat_new = np.zeros((Nx, Np), dtype=complex)
omega = np.zeros((Np, Ny), dtype=float)
#omega_kx = np.zeros((Np, Ny), dtype=float)
#omega_ky = np.zeros((Np, Ny), dtype=float)
#v_grad_omega = np.zeros((Np, Ny), dtype=float)
#psi_hat = np.zeros((Nx, Np), dtype=complex)
#rhs_hat = np.zeros((Nx, Np), dtype=complex)
#rhs = np.zeros((Np, Nx), dtype=float)
#visc_term_complex = np.zeros((Ny, Np), dtype=complex)
#visc_term_real = np.zeros((Np, Ny), dtype=float)
#v_grad_omega_hat = np.zeros((Ny, Np), dtype=complex)
#u_storage_init = np.empty((1, Nx, Nx), dtype=float)
#v_storage_init = np.empty((1, Nx, Nx), dtype=float)
#omega_storage_init = np.empty((1, Nx, Nx), dtype=float)
'''
a = [1. / 6., 1. / 3., 1. / 3., 1. / 6.]
b = [0.5, 0.5, 1.]
'''

# MAIN LOOP:
#t = 0
step = 1                                                  #Substituit isso por n
pbar = tqdm(total=int(n_steps))

# Initial vorticity field
omega_hat_t0 = IC_2D(Nx, Ny, Np, u, v, u_hat, v_hat, IC_type, omega, omega_hat, X, Y)
omega = ifftn_mpi(omega_hat_t0, omega)

# Plot the initial state of the simulation:
# Obs.: np.sqrt(u**2 + v**2) is the amplitude of the velocity and ifftn_mpi(omega_hat, omega) is the curl
plot(0, u, v, np.sqrt(u**2 + v**2), ifftn_mpi(omega_hat, omega), 0, t_f, step, n_steps)

for n in range(n_steps + 1):

    if n == 0: # Initialize vorticity field (for the simulation)
        omega_hat = omega_hat_t0

    # Compute velocity in Fourier space
    u_hat = -iky_over_K2*omega_hat
    v_hat = ikx_over_K2*omega_hat
    u = ifftn_mpi(u_hat, u)
    v = ifftn_mpi(v_hat, v)

    # Compute terms for vorticity equation
    omega_kx = ifftn_mpi(1j*Kx * omega_hat, omega_kx)
    omega_ky = ifftn_mpi(1j*Ky * omega_hat, omega_ky)
    v_grad_omega = (u * omega_kx + v * omega_ky)
    v_grad_omega_hat = fftn_mpi(v_grad_omega, v_grad_omega_hat)*dealias
    visc_term_complex = -viscosity * K2 * omega_hat

    #rhs_hat = visc_term_complex - u_hat * 1j * Kx * omega_hat * dealias - v_hat * 1j * \
    #Ky * omega_hat * dealias
    #rhs_hat = visc_term_complex - v_grad_omega_hat
    #rhs = ifftn_mpi(rhs_hat, rhs)

    # Compute new vorticity field
    omega_hat_new = 1/(1/dt - 0.5*viscosity*LapHat)*(
                (1/dt + 0.5*viscosity*LapHat)*omega_hat - v_grad_omega_hat);
    omega_hat = omega_hat_new.copy()

    '''
    if n%100==0:
        omega = ifftn_mpi(omega_hat,omega)
        plt.imshow(abs(omega),cmap='jet')
        #plt.imshow(sqrt((u ** 2) + (v ** 2)), cmap='jet', animated=True)
        plt.show()
    '''
    '''
    omega_hat1 = omega_hat0 = omega_hat
    for rk in range(4):
        if rk < 3: omega_hat = omega_hat0 + b[rk] * dt * rhs_hat
        omega_hat1 += a[rk] * dt * rhs_hat
    omega_hat = omega_hat1
    '''

    # Plot/Save the vorticity field at intervals
    if step%150==0:
        plot(0, u, v, np.sqrt(u**2 + v**2), ifftn_mpi(omega_hat, omega), n*dt, t_f, step, n_steps)

    #t = t+dt
    step += 1

    pbar.update(1)

pbar.close()

#print('Finished! Time:', time.time()-start_time)


num_processes 1 rank 0


  0%|          | 0/60000 [00:00<?, ?it/s]

NameError: name 'Np' is not defined

# **Backup:**